### 날씨 데이터 파싱
* [기상청날씨데이터](http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp)
* 파싱한 데이터를 dict, list 자료구조에 저장
* 자료구조의 데이터를 json 파일로 저장
* BeautifulSoup 의 find(), find_all(), select(), select_one() 함수 비교

In [ ]:
!pip3 show lxml

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp'

res = requests.get(url)
print(res.status_code)

if res.ok:
    soup = BeautifulSoup(res.text, features='xml')
    # tag 1개
    title_tag = soup.find('title')
    print(type(title_tag), title_tag, title_tag.text)

    # tag 여러개
    title_all_tag = soup.find_all('title')
    print(type(title_all_tag), title_all_tag)
    
    # tag 1개
    location_tag = soup.find('location', attrs={'wl_ver':3})
    print('==find() ', type(location_tag))
    
    # tag 1개
    location_tag2 = soup.select_one("location[wl_ver='3']")
    print('==select_one() ',type(location_tag2))

    # tag 여러개
    loc_tag_all = soup.findAll('location', attrs={'wl_ver':3})
    print('==find_all() ', type(loc_tag_all), len(loc_tag_all))
    
    # tag 여러개
    loc_tag_all2 = soup.select("location[wl_ver='3']")
    print('==select() ', type(loc_tag_all2))

In [ ]:
'''
<location wl_ver="3">
    <province>서울ㆍ인천ㆍ경기도</province>
    <city>서울</city>
    <data>
        <mode>A02</mode>
        <tmEf>2022-08-28 00:00</tmEf>
        <wf>맑음</wf>
        <tmn>18</tmn>
        <tmx>28</tmx>
        <reliability/>
        <rnSt>0</rnSt>
    </data>
    <data>
        <mode>A02</mode>
        <tmEf>2022-08-28 12:00</tmEf>
        <wf>구름많음</wf>
        <tmn>18</tmn>
        <tmx>28</tmx>
        <reliability/>
        <rnSt>30</rnSt>
    </data>
</location>

{
  "province":"서울ㆍ인천ㆍ경기도",
  "city":"서울",
  "datas":[{"mode:"A02”,"tmEf”:"2022-01-31 00:00”,"wf":"맑음"},
            {"mode:"A02”,"tmEf”:"2022-01-31 12:00”,"wf":"구름많음"},{},{}]
}

'''

### 서울(city)의 날씨 데이터 Parsing

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp'

res = requests.get(url)
if res.ok:
    soup = BeautifulSoup(res.text, features='xml')
    location_tag = soup.find('location', attrs={'wl_ver':3})
    location_dict = {}

    location_dict['province'] = location_tag.find('province').text
    location_dict['city'] = location_tag.find('city').text

    data_tags = location_tag.findAll('data')
    data_list = []
    for data_tag in data_tags:
        data_dict = dict()
        data_dict['mode'] = data_tag.find('mode').text
        data_dict['tmEf'] = data_tag.find('tmEf').text
        data_dict['wf'] = data_tag.find('wf').text
        data_dict['tmn'] = data_tag.find('tmn').text
        data_dict['tmx'] = data_tag.find('tmx').text
        data_list.append(data_dict)
        
    location_dict['datas'] = data_list
    print(location_dict)

### 41개 City의  날씨 데이터 파싱

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp'

res = requests.get(url)
if res.ok:
    soup = BeautifulSoup(res.text, features="xml")
    location_tags = soup.find_all('location', attrs={'wl_ver':3})
    # print(len(location_tags))
    location_list = list()
    
    for location_tag in location_tags:    
        location_dict = {} #dict()

        # "province":"서울ㆍ인천ㆍ경기도"
        location_dict['province'] = location_tag.find('province').text
        # 서울
        location_dict['city'] = location_tag.find('city').text

        data_tags = location_tag.find_all('data')
        data_list = []
        for data_tag in data_tags:
            data_dict = dict()
            # {"mode:"A02","tmEf":"2022-01-31 00:00","wf":"맑음"}
            data_dict['mode'] = data_tag.find('mode').text
            data_dict['tmEf'] = data_tag.find('tmEf').text
            data_dict['wf'] = data_tag.find('wf').text
            data_dict['tmn'] = data_tag.find('tmn').text
            data_dict['tmx'] = data_tag.find('tmx').text
            data_list.append(data_dict)

        location_dict['datas'] = data_list
        
        location_list.append(location_dict)
        
print(len(location_list))
#print(location_list)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#### weather.json 파일로 저장하기

In [ ]:
import json

with open('data/weather.json','w',encoding='utf-8') as file:
    json.dump(location_list, file)

#### weather.json 파일을 읽어오기

In [ ]:
import json

with open('data/weather.json',encoding='utf-8') as file:
    json_data = json.load(file)

#print(json_data)